In [1]:
!pip install mediapipe opencv-python numpy

In [2]:
!pip uninstall -y mediapipe
!pip install mediapipe==0.10.8

Found existing installation: mediapipe 0.10.8
Uninstalling mediapipe-0.10.8:
  Successfully uninstalled mediapipe-0.10.8
  Using cached mediapipe-0.10.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached mediapipe-0.10.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)


# 1. Imports y configuración inicial

In [3]:
from mediapipe.python.solutions.hands import Hands, HAND_CONNECTIONS
import mediapipe.python.solutions.drawing_utils as mp_drawing
import cv2
import numpy as np

# Luego creas tu objeto Hands igual...
hands = Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)



# 2. Funciones de análisis de gestos

In [4]:
def count_extended_fingers(lms):
    """
    Cuenta cuántos dedos están extendidos.
    lms: hand_landmarks de MediaPipe
    """
    tips = [4, 8, 12, 16, 20]  # pulgar, índice, medio, anular, meñique
    count = 0

    # Dedos (no pulgar): la punta debe estar "más arriba" que la articulación de dos niveles atrás
    for tip in tips[1:]:
        if lms.landmark[tip].y < lms.landmark[tip-2].y:
            count += 1

    # Pulgar: en horizontal. Para mano derecha, x(4) < x(3) significa extendido
    if lms.landmark[4].x < lms.landmark[3].x:
        count += 1

    return count

def calc_distance(lm1, lm2, w, h):
    """
    Calcula la distancia euclidiana entre dos landmarks normalizados
    y retorna también sus coordenadas absolutas.
    """
    x1, y1 = int(lm1.x * w), int(lm1.y * h)
    x2, y2 = int(lm2.x * w), int(lm2.y * h)
    dist = np.hypot(x2 - x1, y2 - y1)
    return dist, (x1, y1), (x2, y2)


# 3. Loop principal: captura + detección de manos

In [5]:
cap = cv2.VideoCapture(0)
scene = 0
bg_color = (30, 30, 30)      # color de fondo inicial
last_switch = 0             # para debounce

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    # Preparar "lienzo" de salida
    canvas = np.zeros_like(frame)
    canvas[:] = bg_color

    idx_pos = None
    fingers = 0
    pinch_dist = None

    if result.multi_hand_landmarks:
        hand = result.multi_hand_landmarks[0]
        mp_drawing.draw_landmarks(canvas, hand, mp_hands.HAND_CONNECTIONS)

        # 1) Conteo de dedos
        fingers = count_extended_fingers(hand)
        cv2.putText(canvas, f"Dedos: {fingers}", (10,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        # 2) Distancia entre índice (8) y pulgar (4)
        dist, p1, p2 = calc_distance(hand.landmark[8], hand.landmark[4], w, h)
        pinch_dist = int(dist)
        cv2.line(canvas, p1, p2, (0,255,0), 2)
        cv2.putText(canvas, f"Dist: {pinch_dist}", (10,70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        # 3) Posición índice para mover objeto
        idx_pos = p1  # posición de la punta del índice

        # 4) Gestos de acción:
        now = cv2.getTickCount() / cv2.getTickFrequency()

        # — Cambiar escena con palma abierta (5 dedos)
        if fingers == 5 and now - last_switch > 1.0:
            scene = (scene + 1) % 3
            last_switch = now

        # — Cambiar color de fondo con puño cerrado (0 dedos)
        if fingers == 0:
            bg_color = tuple(np.random.randint(50,256,3).tolist())

    # 5) Dibujar objeto que sigue al índice
    if idx_pos:
        cv2.circle(canvas, idx_pos, 40, (255,255,255), -1)

    # 6) Mostrar número de escena
    cv2.putText(canvas, f"Scene: {scene}", (w-200,30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    cv2.imshow("Manos + Gestos", canvas)
    if cv2.waitKey(1) & 0xFF == 27:  # Esc para salir
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.11.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
